In [2]:
import numpy as np 
import pandas as pd
import os

print(os.getcwd())
os.chdir('../../Data/')


c:\Users\chris.slim\OneWorkplace\TRACK Australia - General\Clients & Projects\Volkswagen Group Australia\Volkswagen\Data and Tech\ID Series Brief\4. POC Update\Analysis\Python


In [3]:
import openpyxl as xl



file_mb = '1_inputs/geos/MB_2021_AUST.xlsx'
wb = xl.load_workbook(file_mb) 
print(wb.sheetnames) 

['MB_2021_AUST']


In [4]:
sheet_name_mb = wb.sheetnames[0]
mb = pd.read_excel(file_mb, sheet_name = sheet_name_mb, usecols=['MB_CODE_2021','SA1_CODE_2021','SA2_CODE_2021','SA3_CODE_2021','SA4_CODE_2021',
                                                           'SA2_NAME_2021','SA3_NAME_2021','SA4_NAME_2021',
                                                           'GCCSA_CODE_2021','GCCSA_NAME_2021'])



In [5]:
file_poa = '1_inputs/geos/POA_2021_AUST.xlsx'
sheet_name_poa = 'POA_2021_AUST'
poa = pd.read_excel(file_poa, sheet_name = sheet_name_poa, usecols = ['POA_CODE_2021','MB_CODE_2021'])

In [6]:
mb['sorter'] = mb.groupby("GCCSA_NAME_2021")['MB_CODE_2021'].transform('count')
counts = mb.groupby(['GCCSA_NAME_2021','SA4_NAME_2021','sorter'], as_index=False)['MB_CODE_2021'].count().sort_values('sorter', ascending=False)

#display(counts.iloc[0:50])
#display(counts.iloc[50:100])


ev_cities = mb[(mb['GCCSA_NAME_2021'].isin(['Greater Sydney'
                                             , 'Greater Melbourne'
                                             , 'Greater Brisbane'
                                             , 'Greater Perth'
                                             , 'Greater Darwin'
                                             , 'Greater Adelaide'
                                             , 'Greater Hobart'
                                             ,'Australian Capital Territory'])) | (mb['SA4_NAME_2021'].isin(['Gold Coast'
                                                                                                            ,'Sunshine Coast'
                                                                                                            ,'Newcastle and Lake Macquarie'
                                                                                                            , 'Latrobe - Gippsland']))]

ev_postcodes = pd.merge(left=ev_cities, right=poa, left_on='MB_CODE_2021', right_on='MB_CODE_2021', how='inner').drop('sorter', axis=1)

ev_postcodes.head()

,MB_CODE_2021,SA1_CODE_2021,SA2_CODE_2021,SA2_NAME_2021,SA3_CODE_2021,SA3_NAME_2021,SA4_CODE_2021,SA4_NAME_2021,GCCSA_CODE_2021,GCCSA_NAME_2021,POA_CODE_2021
0,10011670000,12003139338,120031393,Croydon Park - Enfield,12003,Strathfield - Burwood - Ashfield,120,Sydney - Inner West,1GSYD,Greater Sydney,2133
1,10011680000,12003167905,120031679,Croydon,12003,Strathfield - Burwood - Ashfield,120,Sydney - Inner West,1GSYD,Greater Sydney,2132
2,10011690000,12003139338,120031393,Croydon Park - Enfield,12003,Strathfield - Burwood - Ashfield,120,Sydney - Inner West,1GSYD,Greater Sydney,2133
3,10011700000,12003167722,120031677,Ashfield - South,12003,Strathfield - Burwood - Ashfield,120,Sydney - Inner West,1GSYD,Greater Sydney,2131
4,10011710000,12003167727,120031677,Ashfield - South,12003,Strathfield - Burwood - Ashfield,120,Sydney - Inner West,1GSYD,Greater Sydney,2131


In [22]:
ev_postcodes['METRO_AREA'] = np.where(ev_postcodes['GCCSA_NAME_2021'].str.lower().str[0:4] == 'rest', ev_postcodes['SA4_NAME_2021'], ev_postcodes['GCCSA_NAME_2021'])

post_code_area_lkp = ev_postcodes.groupby(['POA_CODE_2021', 'SA3_CODE_2021', 'SA3_NAME_2021','GCCSA_NAME_2021', 'METRO_AREA'],as_index=False)['SA4_CODE_2021'].count().drop('SA4_CODE_2021',axis=1)

unq = post_code_area_lkp.groupby('POA_CODE_2021', as_index=False)['SA3_CODE_2021'].min()

post_code_area_lkp_unq = pd.merge(unq, post_code_area_lkp, how='left', on=['POA_CODE_2021','SA3_CODE_2021'])

print(post_code_area_lkp_unq.shape)

post_code_area_lkp_unq.to_csv('3_outputs/post_code_by_metro_area.csv')


(1219, 5)


In [23]:
post_code_area_lkp_unq.METRO_AREA.value_counts()

METRO_AREA
Greater Melbourne               284
Greater Sydney                  259
Greater Adelaide                151
Greater Brisbane                142
Greater Perth                   130
Latrobe - Gippsland              89
Greater Hobart                   38
Newcastle and Lake Macquarie     34
Sunshine Coast                   27
Australian Capital Territory     26
Gold Coast                       23
Greater Darwin                   16
Name: count, dtype: int64

In [25]:
print(post_code_area_lkp_unq[25:40])

   POA_CODE_2021 SA3_CODE_2021            SA3_NAME_2021 GCCSA_NAME_2021  \
25          2018         11701                   Botany  Greater Sydney   
26          2019         11701                   Botany  Greater Sydney   
27          2020         11701                   Botany  Greater Sydney   
28          2021         11801  Eastern Suburbs - North  Greater Sydney   
29          2022         11801  Eastern Suburbs - North  Greater Sydney   
30          2023         11801  Eastern Suburbs - North  Greater Sydney   
31          2024         11801  Eastern Suburbs - North  Greater Sydney   
32          2025         11801  Eastern Suburbs - North  Greater Sydney   
33          2026         11801  Eastern Suburbs - North  Greater Sydney   
34          2027         11801  Eastern Suburbs - North  Greater Sydney   
35          2028         11801  Eastern Suburbs - North  Greater Sydney   
36          2029         11801  Eastern Suburbs - North  Greater Sydney   
37          2030         

: 